In [1023]:
# %pylab inline
import numpy as np
import matplotlib.pyplot as plt
# %config Completer.use_jedi = False

In [1024]:
def gamma_to_beta(gamma):
    return sqrt(1 - 1 / gamma**2)


def kinetic_to_betagamma(energy, rest_energy):
    gamma = energy / rest_energy + 1
    beta = gamma_to_beta(gamma)
    return beta, gamma

In [1025]:
def bouncer_flat(vector, wall_Z, apt_R):
    """
    taking losses at the circular aperture and generating a full image bunch reflecting off a flat wall

    includes cutoffs for particles striking wall or passing through the aperture
    """
    result = {}
    result["x"] = np.zeros_like(vector["x"])
    result["y"] = np.zeros_like(vector["y"])
    result["z"] = np.zeros_like(vector["z"])
    result["t"] = np.zeros_like(vector["t"])
    result["Px"] = np.zeros_like(vector["Px"])
    result["Py"] = np.zeros_like(vector["Py"])
    result["Pz"] = np.zeros_like(vector["Pz"])
    result["Pt"] = np.zeros_like(vector["Pt"])
    result["gamma"] = np.zeros_like(vector["gamma"])
    result["bx"] = np.zeros_like(vector["bx"])
    result["by"] = np.zeros_like(vector["by"])
    result["bz"] = np.zeros_like(vector["bz"])
    result["bdotx"] = np.zeros_like(vector["bdotx"])
    result["bdoty"] = np.zeros_like(vector["bdoty"])
    result["bdotz"] = np.zeros_like(vector["bdotz"])
    result["q"] = vector["q"]
    for i in range(len(vector["x"])):
        r = np.sqrt(vector["x"][i] ** 2 + vector["y"][i] ** 2)
        # turning off images for particles passing the wall
        if vector["z"][i] >= wall_Z:
            result["q"] = 0
            # break
        # vector['z'][i]<wall_Z and r<=apt_R:
        else:
            result["q"] = -vector["q"] * macro_pop
        result["z"][i] = wall_Z + 2 * (wall_Z - vector["z"][i])
        result["x"][i] = vector["x"][i]
        result["y"][i] = vector["y"][i]
        result["Px"][i] = vector["Px"][i] * macro_pop
        result["Py"][i] = vector["Py"][i] * macro_pop
        result["Pz"][i] = -vector["Pz"][i] * macro_pop
        result["Pt"][i] = vector["Pt"][i] * macro_pop  # right?
        result["gamma"][i] = vector["gamma"][i] / macro_pop
        result["bx"][i] = vector["bx"][i] / macro_pop
        result["by"][i] = vector["by"][i] / macro_pop
        result["bz"][i] = -vector["bz"][i] / macro_pop
        result["bdotx"][i] = vector["bdotx"][i]
        result["bdoty"][i] = vector["bdoty"][i]
        result["bdotz"][i] = -vector["bdotz"][i]
        result["t"][i] = vector["t"][
            i
        ]  # do NOT retard here, image charge is made to exist at the moment the original charge is created

    return result

In [1026]:
def dist_euclid(vector, vector_ext, index):
    """
    simple Euclidean distance generator

    """
    result = {}
    result["R"] = np.zeros_like(vector["x"])
    result["nx"] = np.zeros_like(vector["x"])
    result["ny"] = np.zeros_like(vector["x"])
    result["nz"] = np.zeros_like(vector["x"])
    for j in range(len(vector_ext["x"])):
        result["R"][j] = np.sqrt(
            (vector["x"][index] - vector_ext["x"][j]) ** 2
            + (vector["y"][index] - vector_ext["y"][j]) ** 2
            + (vector["z"][index] - vector_ext["z"][j]) ** 2
        )
        result["nx"][j] = (vector["x"][index] - vector_ext["x"][j]) / result["R"][j]
        result["ny"][j] = (vector["y"][index] - vector_ext["y"][j]) / result["R"][j]
        result["nz"][j] = (vector["z"][index] - vector_ext["z"][j]) / result["R"][j]
    return result

In [1027]:
def dist_euclid_ret(trajectory, trajectory_ext, index_traj, index_part, indices_ret):
    """
    simple Euclidean distance generator

    """
    result = {}
    result["R"] = np.zeros_like(trajectory[index_traj]["x"])
    result["nx"] = np.zeros_like(trajectory[index_traj]["x"])
    result["ny"] = np.zeros_like(trajectory[index_traj]["x"])
    result["nz"] = np.zeros_like(trajectory[index_traj]["x"])
    for j in range(len(trajectory[index_traj]["x"])):
        result["R"][j] = np.sqrt(
            (
                trajectory[index_traj]["x"][index_part]
                - trajectory_ext[indices_ret[j]]["x"][j]
            )
            ** 2
            + (
                trajectory[index_traj]["y"][index_part]
                - trajectory_ext[indices_ret[j]]["y"][j]
            )
            ** 2
            + (
                trajectory[index_traj]["z"][index_part]
                - trajectory_ext[indices_ret[j]]["z"][j]
            )
            ** 2
        )
        result["nx"][j] = (
            trajectory[index_traj]["x"][index_part]
            - trajectory_ext[indices_ret[j]]["x"][j]
        ) / result["R"][j]
        result["ny"][j] = (
            trajectory[index_traj]["y"][index_part]
            - trajectory_ext[indices_ret[j]]["y"][j]
        ) / result["R"][j]
        result["nz"][j] = (
            trajectory[index_traj]["z"][index_part]
            - trajectory_ext[indices_ret[j]]["z"][j]
        ) / result["R"][j]
    return result

In [1028]:
def eqsofmotion_static(
    h, vector, vector_ext
):  # nhat includes R and fnhat components, need to generate this per particle pair
    result = {}
    result["x"] = np.zeros_like(vector["x"])
    result["y"] = np.zeros_like(vector["y"])
    result["z"] = np.zeros_like(vector["z"])
    result["t"] = np.zeros_like(vector["t"])
    result["Px"] = np.zeros_like(vector["Px"])
    result["Py"] = np.zeros_like(vector["Py"])
    result["Pz"] = np.zeros_like(vector["Pz"])
    result["Pt"] = np.zeros_like(vector["Pt"])
    result["gamma"] = np.zeros_like(vector["gamma"])
    result["bx"] = np.zeros_like(vector["bx"])
    result["by"] = np.zeros_like(vector["by"])
    result["bz"] = np.zeros_like(vector["bz"])
    result["bdotx"] = np.zeros_like(vector["bdotx"])
    result["bdoty"] = np.zeros_like(vector["bdoty"])
    result["bdotz"] = np.zeros_like(vector["bdotz"])
    result["q"] = vector["q"]
    for i in range(
        len(vector["x"])
    ):  # iterating over all real particles OR all reflection points (these must be done in separate steps)
        nhat = dist_euclid(vector, vector_ext, i)
        for j in range(
            len(vector_ext["x"])
        ):  # summing all external contributions (reflected particles and/or local particles)
            # b_nhat = vector['bx'][i]*nhat['nx'][j]+vector['by'][i]*nhat['ny'][j]+vector['bz'][i]*nhat['nz'][j] #for accurate chrono-matching
            result["x"][i] = vector["x"][i]
            result["y"][i] = vector["y"][i]
            result["z"][i] = vector["z"][i]
            result["t"][i] = vector["t"][i]
            result["Px"][i] = vector["Px"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["bx"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["nx"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nx"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Py"][i] = vector["Py"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["by"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["ny"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["ny"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Pz"][i] = vector["Pz"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["bz"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["nz"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nz"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Pt"][i] = vector["Pt"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * (
                (
                    (
                        vector_ext["gamma"][j] ** 2
                        * np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            ),
                        )
                    )
                    / (c * nhat["R"][j])
                    - 1 / nhat["R"][j] ** 2
                )
                / (
                    1
                    - np.dot(
                        (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                )
            )
            result["gamma"][i] = result["Pt"][i] / (m * c)
            result["bx"][i] = result["Px"][i] / (m * c * result["gamma"][i])
            result["by"][i] = result["Py"][i] / (m * c * result["gamma"][i])
            result["bz"][i] = result["Pz"][i] / (m * c * result["gamma"][i])
            result["bdotx"][i] = (result["Px"][i] - vector["Px"][i]) / (
                m * c
            )  # use 1/h???
            result["bdoty"][i] = (result["Py"][i] - vector["Py"][i]) / (m * c)
            result["bdotz"][i] = (result["Pz"][i] - vector["Pz"][i]) / (m * c)

            # NOTE---- Momentum values below are updated 'result', implicit technically, but only needs explicit solver for extreme
            result["x"][i] = vector["x"][i] + h / m * (
                result["Px"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["bx"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["y"][i] = vector["y"][i] + h / m * (
                result["Py"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["by"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["z"][i] = vector["z"][i] + h / m * (
                result["Pz"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["bz"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["t"][i] = vector["t"][i] + h / m * (
                result["Pt"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
    return result

In [1029]:
def static_integrator(steps, h_step, bunch_real, wall_Z, apt_R):
    trajectory = [{}] * steps
    trajectory_img = [{}] * steps
    for i in range(steps):
        if i == 0:
            trajectory[i] = init_beam
            trajectory_img[i] = bouncer_flat(
                init_beam, wall_Z, apt_R
            )  # note that init_wall is a dummy vector
        else:
            trajectory[i] = eqsofmotion_static(
                h_step, trajectory[i - 1], trajectory_img[i - 1]
            )
            trajectory_img[i] = bouncer_flat(
                trajectory[i], wall_Z, apt_R
            )  # note that init_wall is a dummy vector
    return trajectory, trajectory_img

In [1030]:
def chrono_jn(trajectory, trajectory_ext, index_traj, index_part):
    nhat = dist_euclid(trajectory[index_traj], trajectory_ext[index_traj], index_part)
    index_traj_new = np.empty(len(trajectory_ext[index_traj]["x"]), dtype=int)
    for l in range(len(trajectory_ext[index_traj]["x"])):
        b_nhat = (
            trajectory_ext[index_traj]["bx"][l] * nhat["nx"][l]
            + trajectory_ext[index_traj]["by"][l] * nhat["ny"][l]
            + trajectory_ext[index_traj]["bz"][l] * nhat["nz"][l]
        )  # for accurate chrono-matching
        # b_nhat = trajectory[index_traj]['bz'][index_part]*nhat['nz'][l] #for speedup
        delta_t = nhat["R"][l] * (1 + b_nhat) / c
        t_ext_new = trajectory_ext[index_traj]["t"][l] - delta_t
        if t_ext_new < 0:
            index_traj_new[l] = index_traj
        else:
            for k in range(index_traj, -1, -1):
                if trajectory_ext[index_traj - k]["t"][l] > t_ext_new:
                    index_traj_new[l] = index_traj - k
                    break
    return index_traj_new

In [1031]:
def eqsofmotion_retarded(
    h, trajectory, trajectory_ext, apt_R
):  # nhat includes R and fnhat components, need to generate this per particle pair
    result = {}
    result["x"] = np.zeros_like(trajectory[0]["x"])
    result["y"] = np.zeros_like(trajectory[0]["y"])
    result["z"] = np.zeros_like(trajectory[0]["z"])
    result["t"] = np.zeros_like(trajectory[0]["t"])
    result["Px"] = np.zeros_like(trajectory[0]["Px"])
    result["Py"] = np.zeros_like(trajectory[0]["Py"])
    result["Pz"] = np.zeros_like(trajectory[0]["Pz"])
    result["Pt"] = np.zeros_like(trajectory[0]["Pt"])
    result["gamma"] = np.zeros_like(trajectory[0]["gamma"])
    result["bx"] = np.zeros_like(trajectory[0]["bx"])
    result["by"] = np.zeros_like(trajectory[0]["by"])
    result["bz"] = np.zeros_like(trajectory[0]["bz"])
    result["bdotx"] = np.zeros_like(trajectory[0]["bdotx"])
    result["bdoty"] = np.zeros_like(trajectory[0]["bdoty"])
    result["bdotz"] = np.zeros_like(trajectory[0]["bdotz"])
    result["q"] = trajectory[0]["q"]
    for i in range(len(trajectory)):
        for l in range(
            len(trajectory[0]["x"])
        ):  # iterating over all real particles OR all reflection points (these must be done in separate steps)
            i_new = chrono_jn(
                trajectory, trajectory_ext, i, l
            )  # notes: i_new is a list corresponding to the j index.... also, an intermediate nhat is being calculated here
            nhat = dist_euclid_ret(
                trajectory, trajectory_ext, i, l, i_new
            )  # yes, we need a new nhat now
            for j in range(
                len(trajectory_ext[0]["x"])
            ):  # summing all external contributions (reflected particles and/or local particles)
                if nhat["R"][j] < 1.5 * apt_R:
                    trajectory_ext[i_new[j]]["q"] = 0
                result["x"][l] = trajectory[i]["x"][l]
                result["y"][l] = trajectory[i]["y"][l]
                result["z"][l] = trajectory[i]["z"][l]
                result["t"][l] = trajectory[i]["t"][l]
                result["Px"][l] = trajectory[i]["Px"][l] + h / m * trajectory[i][
                    "q"
                ] * trajectory_ext[i_new[j]]["q"] * trajectory[i]["gamma"][l] * 1 / (
                    1
                    - np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                    ** 2
                ) * (
                    1
                    / nhat["R"][j] ** 2
                    * trajectory[i]["bx"][l]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    - nhat["nx"][j]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j] ** 2
                        * nhat["R"][j] ** 2
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                    - nhat["nx"][j]
                    / (c * nhat["R"][j])
                    * (
                        np.dot(
                            (
                                trajectory[i]["bx"][l],
                                trajectory[i]["by"][l],
                                trajectory[i]["bz"][l],
                            ),
                            (
                                (
                                    trajectory_ext[i_new[j]]["bdotx"][j],
                                    trajectory_ext[i_new[j]]["bdoty"][j],
                                    trajectory_ext[i_new[j]]["bdotz"][j],
                                )
                                + trajectory_ext[i_new[j]]["gamma"][j] ** 2
                                * np.dot(
                                    (
                                        trajectory_ext[i_new[j]]["bx"][j],
                                        trajectory_ext[i_new[j]]["by"][j],
                                        trajectory_ext[i_new[j]]["bz"][j],
                                    ),
                                    np.dot(
                                        (
                                            trajectory_ext[i_new[j]]["bx"][j],
                                            trajectory_ext[i_new[j]]["by"][j],
                                            trajectory_ext[i_new[j]]["bz"][j],
                                        ),
                                        (
                                            trajectory_ext[i_new[j]]["bdotx"][j],
                                            trajectory_ext[i_new[j]]["bdoty"][j],
                                            trajectory_ext[i_new[j]]["bdotz"][j],
                                        ),
                                    ),
                                )
                            ),
                        )
                    )
                )
                result["Py"][l] = trajectory[i]["Py"][l] + h / m * trajectory[i][
                    "q"
                ] * trajectory_ext[i_new[j]]["q"] * trajectory[i]["gamma"][l] * 1 / (
                    1
                    - np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                    ** 2
                ) * (
                    1
                    / nhat["R"][j] ** 2
                    * trajectory[i]["by"][l]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    - nhat["ny"][j]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j] ** 2
                        * nhat["R"][j] ** 2
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                    - nhat["ny"][j]
                    / (c * nhat["R"][j])
                    * (
                        np.dot(
                            (
                                trajectory[i]["bx"][l],
                                trajectory[i]["by"][l],
                                trajectory[i]["bz"][l],
                            ),
                            (
                                (
                                    trajectory_ext[i_new[j]]["bdotx"][j],
                                    trajectory_ext[i_new[j]]["bdoty"][j],
                                    trajectory_ext[i_new[j]]["bdotz"][j],
                                )
                                + trajectory_ext[i_new[j]]["gamma"][j] ** 2
                                * np.dot(
                                    (
                                        trajectory_ext[i_new[j]]["bx"][j],
                                        trajectory_ext[i_new[j]]["by"][j],
                                        trajectory_ext[i_new[j]]["bz"][j],
                                    ),
                                    np.dot(
                                        (
                                            trajectory_ext[i_new[j]]["bx"][j],
                                            trajectory_ext[i_new[j]]["by"][j],
                                            trajectory_ext[i_new[j]]["bz"][j],
                                        ),
                                        (
                                            trajectory_ext[i_new[j]]["bdotx"][j],
                                            trajectory_ext[i_new[j]]["bdoty"][j],
                                            trajectory_ext[i_new[j]]["bdotz"][j],
                                        ),
                                    ),
                                )
                            ),
                        )
                    )
                )
                result["Pz"][l] = trajectory[i]["Pz"][l] + h / m * trajectory[i][
                    "q"
                ] * trajectory_ext[i_new[j]]["q"] * trajectory[i]["gamma"][l] * 1 / (
                    1
                    - np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                    ** 2
                ) * (
                    1
                    / nhat["R"][j] ** 2
                    * trajectory[i]["bz"][l]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    - nhat["nz"][j]
                    * np.dot(
                        (
                            trajectory_ext[i_new[j]]["bx"][j],
                            trajectory_ext[i_new[j]]["by"][j],
                            trajectory_ext[i_new[j]]["bz"][j],
                        ),
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                    )
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j] ** 2
                        * nhat["R"][j] ** 2
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                    - nhat["nz"][j]
                    / (c * nhat["R"][j])
                    * (
                        np.dot(
                            (
                                trajectory[i]["bx"][l],
                                trajectory[i]["by"][l],
                                trajectory[i]["bz"][l],
                            ),
                            (
                                (
                                    trajectory_ext[i_new[j]]["bdotx"][j],
                                    trajectory_ext[i_new[j]]["bdoty"][j],
                                    trajectory_ext[i_new[j]]["bdotz"][j],
                                )
                                + trajectory_ext[i_new[j]]["gamma"][j] ** 2
                                * np.dot(
                                    (
                                        trajectory_ext[i_new[j]]["bx"][j],
                                        trajectory_ext[i_new[j]]["by"][j],
                                        trajectory_ext[i_new[j]]["bz"][j],
                                    ),
                                    np.dot(
                                        (
                                            trajectory_ext[i_new[j]]["bx"][j],
                                            trajectory_ext[i_new[j]]["by"][j],
                                            trajectory_ext[i_new[j]]["bz"][j],
                                        ),
                                        (
                                            trajectory_ext[i_new[j]]["bdotx"][j],
                                            trajectory_ext[i_new[j]]["bdoty"][j],
                                            trajectory_ext[i_new[j]]["bdotz"][j],
                                        ),
                                    ),
                                )
                            ),
                        )
                    )
                )
                result["Pt"][l] = trajectory[i]["Pt"][l] + h / m * trajectory[i][
                    "q"
                ] * trajectory_ext[i_new[j]]["q"] * trajectory[i]["gamma"][l] * (
                    (
                        (
                            trajectory_ext[i_new[j]]["gamma"][j] ** 2
                            * np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (
                                    trajectory_ext[i_new[j]]["bdotx"][j],
                                    trajectory_ext[i_new[j]]["bdoty"][j],
                                    trajectory_ext[i_new[j]]["bdotz"][j],
                                ),
                            )
                        )
                        / (c * nhat["R"][j])
                        - 1 / nhat["R"][j] ** 2
                    )
                    / (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i_new[j]]["bx"][j],
                                trajectory_ext[i_new[j]]["by"][j],
                                trajectory_ext[i_new[j]]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                result["gamma"][l] = result["Pt"][l] / (m * c)
                result["bx"][l] = result["Px"][l] / (m * c * result["gamma"][l])
                result["by"][l] = result["Py"][l] / (m * c * result["gamma"][l])
                result["bz"][l] = result["Pz"][l] / (m * c * result["gamma"][l])
                result["bdotx"][l] = (result["Px"][l] - trajectory[i]["Px"][l]) / (
                    m * c
                )  # use 1/h???
                result["bdoty"][l] = (result["Py"][l] - trajectory[i]["Py"][l]) / (
                    m * c
                )
                result["bdotz"][l] = (result["Pz"][l] - trajectory[i]["Pz"][l]) / (
                    m * c
                )

                # NOTE---- Momentum values below are updated 'result', implicit technically, but only needs explicit solver for extreme
                result["x"][l] = trajectory[i]["x"][l] + h / m * (
                    result["Px"][l]
                    + trajectory[i]["q"]
                    / c
                    * trajectory_ext[i_new[j]]["q"]
                    * trajectory_ext[i_new[j]]["bx"][j]
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j]
                        * c
                        * nhat["R"][j]
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                )
                result["y"][l] = trajectory[i]["y"][l] + h / m * (
                    result["Py"][l]
                    + trajectory[i]["q"]
                    / c
                    * trajectory_ext[i_new[j]]["q"]
                    * trajectory_ext[i_new[j]]["by"][j]
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j]
                        * c
                        * nhat["R"][j]
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                )
                result["z"][l] = trajectory[i]["z"][l] + h / m * (
                    result["Pz"][l]
                    + trajectory[i]["q"]
                    / c
                    * trajectory_ext[i_new[j]]["q"]
                    * trajectory_ext[i_new[j]]["bz"][j]
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j]
                        * c
                        * nhat["R"][j]
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                )
                result["t"][l] = trajectory[i]["t"][l] + h / m * (
                    result["Pt"][l]
                    + trajectory[i]["q"]
                    / c
                    * trajectory_ext[i_new[j]]["q"]
                    / (
                        trajectory_ext[i_new[j]]["gamma"][j]
                        * c
                        * nhat["R"][j]
                        * (
                            1
                            - np.dot(
                                (
                                    trajectory_ext[i_new[j]]["bx"][j],
                                    trajectory_ext[i_new[j]]["by"][j],
                                    trajectory_ext[i_new[j]]["bz"][j],
                                ),
                                (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                            )
                        )
                    )
                )
    return result

In [1032]:
def retarded_integrator(steps_init, steps_retarded, h_step, bunch_real, wall_Z, apt_R):
    trajectory, trajectory_img = static_integrator(
        steps_init, h_step, bunch_real, wall_Z, apt_R
    )
    trajectory_new = [{}] * steps_retarded
    trajectory_img_new = [{}] * steps_retarded
    for i in range(steps_retarded):
        if i <= steps_init - 1:
            trajectory_new[i] = trajectory[i]
            trajectory_img_new[i] = trajectory_img[
                i
            ]  # note that init_wall is a dummy vector
        else:
            trajectory_new[i] = eqsofmotion_retarded(
                h_step, trajectory_new[0 : i - 1], trajectory_img_new[0 : i - 1], apt_R
            )
            trajectory_img_new[i] = bouncer_flat(
                trajectory_new[i], wall_Z, apt_R
            )  # note that init_wall is a dummy vector
    return trajectory_new, trajectory_img_new

In [1033]:
pcount = 2  # macroparticle count
c = 299.792458  # mm/ns
macro_pop = 1e9
# m_particle = 0.00055   #electron -amu
m_particle = 1.007319468  # proton -amu
m = m_particle  # *macro_pop #macroparticle mass
q = 1.178734e-5  ##1.6E-19 C => 4.8032047E-10 statC [cm^(3/2)*g^(1/2)*s^(-1)] => [mm^(3/2)*amu^(1/2)*ns^(-1)];
aperture = 3  # mm
wall_pos = 30  # wall location on beam axis 'z', relative to origin


Px = np.random.normal(-3e-2, 3e-2, pcount) * m  # 3e-2 amu*mm/ns corresponds to 93 keV
Py = np.random.uniform(-3e-2, 3e-2, pcount) * m
Pz = (
    np.random.uniform(1e5, 1.05e5, pcount) * m
)  #  approx 1 TeV per particle, CAREFUL WE ARE IN LAB FRAME HERE
# Pz = np.random.uniform(1e3, 2e3, pcount)*m  #  approx 1 TeV per particle, CAREFUL WE ARE IN LAB FRAME HERE
Pt = np.sqrt(Px**2 + Py**2 + Pz**2 + m**2 * c**2)
gamma = Pt / (m * c)
bx = Px / (gamma * m * c)
by = Py / (gamma * m * c)
bz = Pz / (gamma * m * c)
beta_avg = np.sqrt(bx**2 + by**2 + bz**2)

x = np.random.uniform(-0.3, 0.3, pcount)
y = np.random.uniform(-0.3, 0.3, pcount)
z = np.random.uniform(-20, -17, pcount)
# t = np.random.normal(0, 100/(beta_avg*c), pcount) #from our old paper, not sure if this is necessary
t = np.zeros(pcount)

bdotx = np.zeros(pcount)  # bx*np.random.uniform(-8e-2,8e-2)
bdoty = np.zeros(pcount)  # by*np.random.uniform(-8e-2,8e-2)
bdotz = np.zeros(pcount)  # bz*np.random.uniform(-8e-2,8e-2)

In [1034]:
init_beam = {
    "x": x,
    "y": y,
    "z": z,
    "t": t,
    "Px": Px,
    "Py": Py,
    "Pz": Pz,
    "Pt": Pt,
    "bx": bx,
    "by": by,
    "bz": bz,
    "bdotx": bdotx,
    "bdoty": bdoty,
    "bdotz": bdotz,
    "gamma": gamma,
    "q": q,
}

In [ ]:
init_steps = 50
tot_steps = 300
step_size = 3e-6
retarded_traj, retarded_img_traj = retarded_integrator(
    init_steps, tot_steps, step_size, init_beam, wall_pos, aperture
)

In [ ]:
delta_e = [
    np.mean(retarded_traj[i]["gamma"] - init_beam["gamma"]) * m * c**2
    for i in range(2, tot_steps - 1)
]

In [ ]:
delta_e_MeV = np.multiply(delta_e, 938.27)

In [ ]:
zs = [np.mean(retarded_traj[i]["z"]) for i in range(2, tot_steps - 1)]

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(zs, delta_e_MeV, s=40)
ax.set_xlabel("z position")
ax.set_ylabel(r"$\Delta$ E in MeV")
ax.axvline(30, color="g", label="wall position")
plt.legend()
plt.show()

In [1018]:
(retarded_traj[tot_steps - 1]["bz"] - init_beam["bz"])

array([2.90583783e-06, 2.87150384e-06])

In [1019]:
retarded_traj[tot_steps - 1]["bdotx"] - init_beam["bdotx"]

array([0., 0.])

In [1020]:
(retarded_traj[tot_steps - 1]["gamma"] - init_beam["gamma"]) * m * c**2

array([0.01064676, 0.01090856])

In [1021]:
(retarded_traj[tot_steps - 1]["Px"] - init_beam["Px"])

array([-1.57844458e-04,  1.12875681e-12])

In [1022]:
retarded_traj[tot_steps - 1]["gamma"]

array([334.45557879, 347.48144514])